This notebook is used to inspect the verification results running by and only by this kernel user. The user will see all the verification results and will be able to compare the results with the 'standard' library where the pseudopotentials verifications run and already uploaded to Materials Cloud SSSP database.

User can choose his finished verification running process and inspect the outputs by using `WorkChainSelector`. 

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [1]:
import os
import json

import ipywidgets as widgets
from widget_periodictable import PTableWidget
from ipywidgets import Layout

In [2]:
# the mock db is a folder which contains pseudos in the sub-folders named by element seprately.
# This should be some lightweight database constructed from SSSP web page and local-run aiida postgresql database 
sssp_db = './data_mock_sssp/'
local_db = './data_mock_local/'

enable_elements = [e for e in os.listdir(sssp_db) if not e.startswith('.')] + [e for e in os.listdir(local_db) if not e.startswith('.')]

sssp_pseudos = {}
local_pseudos = {}

for e in enable_elements:
    parent_folder = os.path.join(sssp_db, e)
    l = os.listdir(parent_folder)
    sssp_pseudos[e] = [os.path.join(parent_folder, f) for f in l]
    
for e in enable_elements:
    parent_folder = os.path.join(local_db, e)
    l = os.listdir(parent_folder)
    local_pseudos[e] = [os.path.join(parent_folder, f) for f in l]
    
# Show the widget
ptwidget = PTableWidget(states = 1,
                      selected_colors = ['royalblue'], 
                      unselected_color='cornflowerblue', border_color = 'black', width = '20px')

disable_elements = [e for e in ptwidget.allElements if e not in enable_elements]
ptwidget.disabled_elements = disable_elements
display(ptwidget)

PTableWidget(allElements=['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', '…

In [3]:
enable_elements

['Au', 'Si', 'Au', 'Si']

In [4]:
sssp_pseudos

{'Au': ['./data_mock_sssp/Au/Au_sg15_v1.0.json',
  './data_mock_sssp/Au/Au_sg15_v1.2.json',
  './data_mock_sssp/Au/Au_us_gbrv_v1.json'],
 'Si': ['./data_mock_sssp/Si/Si_us_gbrv_v1.0.json',
  './data_mock_sssp/Si/Si_nc_sg15_v1.1.json',
  './data_mock_sssp/Si/Si_paw_psl_v1.0_high.json']}

In [5]:
local_pseudos

{'Au': [],
 'Si': ['./data_mock_local/Si/Si_nc_sg15_v1.2.json',
  './data_mock_local/Si/Si_us_psl_v1.0_high.json',
  './data_mock_local/Si/Si_nc_sg15_v1.0.json']}

In [6]:
ptwidget.selected_elements

{}

In [7]:
import traitlets

class PseudoDBToggleWidget(widgets.VBox):
    
    sssp = traitlets.Bool()
    local = traitlets.Bool()
    
    def __init__(self):
        sssp_db_toggle = widgets.ToggleButton(
            value=False,
            description='SSSP Database',
            disabled=False,
            tooltip='Toggle for SSSP database.',
        )

        local_db_toggle = widgets.ToggleButton(
            value=False,
            description='Local Database',
            disabled=False,
            tooltip='Toggle for Local database.',
        )
        
        sssp_db_toggle.observe(self._sssp_db_toggle, 'value')
        local_db_toggle.observe(self._local_db_toggle, 'value')

        db_choice_widget = widgets.HBox(children=[sssp_db_toggle, local_db_toggle])
        
        super().__init__(
            children=[
                db_choice_widget,
            ]
        )
        
    def _sssp_db_toggle(self, change):
        self.sssp = change['new']
        
    def _local_db_toggle(self, change):
        self.local = change['new']
        
pseudo_db = PseudoDBToggleWidget()
display(pseudo_db)

PseudoDBToggleWidget(children=(HBox(children=(ToggleButton(value=False, description='SSSP Database', tooltip='…

In [8]:
pseudo_db.local

False

In [9]:
class PseudoChooseWidget(widgets.VBox):
    
    sssp_list = traitlets.List()
    local_list = traitlets.List()
    
    output_pseudo_list = traitlets.List()
    
    def __init__(self):
        
        self.out_sssp = []
        self.out_local = []
        
        self.sssp = widgets.SelectMultiple(
            options=self.sssp_list,
            description='SSSP Database',
            disabled=False,
        )
        
        self.sssp.observe(self._update_sssp, 'value')
        
        self.local = widgets.SelectMultiple(
            options=self.local_list,
            description='Local Database',
            disabled=False,
        )
        
        self.local.observe(self._update_local, 'value')
        
        super().__init__(
            children=[
                self.sssp,
                self.local,
            ]
        )
        
    @traitlets.observe('sssp_list')
    def _observe_sssp_list(self, change):
        if change['new']:
            self.sssp.options = change['new']
            
    @traitlets.observe('local_list')
    def _observe_local_list(self, change):
        if change['new']:
            self.local.options = change['new']
            
    def _update_sssp(self, change):
        self.out_sssp = list(change['new'])
        
        self.output_pseudo_list = self.out_sssp + self.out_local
        
    def _update_local(self, change):
        self.out_local = list(change['new'])
        
        self.output_pseudo_list = self.out_sssp + self.out_local

In [10]:
pcw = PseudoChooseWidget()

In [11]:
confirm_button = widgets.Button(
    description='Confirm',
)
output = widgets.Output()

display(confirm_button, output)

def _on_confirm_clicked(b):
    element = list(ptwidget.selected_elements.keys())[0]
    # search sssp db and local db for given element
    if pseudo_db.sssp:
        pcw.sssp_list = sssp_pseudos[element]
    else:
        pcw.sssp_list = []

    if pseudo_db.local:
        pcw.local_list = local_pseudos[element]
    else:
        pcw.local_list = []
    
confirm_button.on_click(_on_confirm_clicked)

Button(description='Confirm', style=ButtonStyle())

Output()

In [12]:
display(pcw)

PseudoChooseWidget(children=(SelectMultiple(description='SSSP Database', options=(), value=()), SelectMultiple…

[5.928629517208, 5.9286099081018, 3.0734163139081, 5.4790266075018]
[5.928629517208, 5.9286099081018, 3.0734163139081, 5.4790266075018]
[5.928629517208, 5.9286099081018, 5.4790266075018]
[5.928629517208, 5.9286099081018, 5.4790266075018]
[5.928629517208, 5.9286099081018, 3.0734163139081, 5.4790266075018]
[5.928629517208, 5.9286099081018, 3.0734163139081, 5.4790266075018]
[2.0896692104089, 5.928629517208, 5.9286099081018, 3.0734163139081, 5.4790266075018]
[2.0896692104089, 5.928629517208, 5.9286099081018, 3.0734163139081, 5.4790266075018]
[2.0896692104089, 5.928629517208, 2.9863598770637, 5.9286099081018, 3.0734163139081, 5.4790266075018]
[2.0896692104089, 5.928629517208, 2.9863598770637, 5.9286099081018, 3.0734163139081, 5.4790266075018]
[2.9863598770637, 5.9286099081018, 3.0734163139081, 5.4790266075018]
Histogram({
    'x': [./data_mock_sssp/Si/Si_paw_psl_v1.0_high.json,
          ./data_mock_local/Si/Si_nc_sg15_v1.2.json,
          ./data_mock_local/Si/Si_us_psl_v1.0_high.json,
    

In [26]:
pcw.output_pseudo_list

['./data_mock_sssp/Si/Si_nc_sg15_v1.1.json',
 './data_mock_local/Si/Si_nc_sg15_v1.2.json',
 './data_mock_local/Si/Si_us_psl_v1.0_high.json',
 './data_mock_local/Si/Si_nc_sg15_v1.0.json']

In [14]:
import plotly.graph_objects as go

In [15]:
def plot_convergence(property_name, dataset):
    traces = []

    for fname in dataset:
        with open(fname, 'rb') as f:
            p = json.load(f)

        co = p[property_name]['output_parameters']
        traces.append(go.Scatter(x=co['ecutwfc'], y=co['relative_diff'],
                            mode='lines+markers',
                            name=os.path.basename(fname)))

    return traces

In [45]:
def plot_delta_hist(dataset):
    
    pp_name = []
    pp_delta = []
    for fname in dataset:
        with open(fname, 'rb') as f:
            p = json.load(f)
            
            pp_delta.append(p['delta_factor']['output_delta_factor']['delta'])
            pp_name.append(fname)
        
    trace = go.Bar(x=pp_name, y=pp_delta)
    return trace

In [48]:
g0 = go.FigureWidget(data=[],
                   layout=go.Layout(
                       title='Delta Factor',
                       xaxis_title='pseudos',
                       yaxis_title='delta',
                   ))

g1 = go.FigureWidget(data=[],
                   layout=go.Layout(
                       title='Cohesive',
                       xaxis_title='ecutwfc',
                       yaxis_title='relative diff(%)',
                   ))

g2 = go.FigureWidget(data=[],
                   layout=go.Layout(
                       title='Phonon',
                       xaxis_title='ecutwfc',
                       yaxis_title='relative diff(%)',
                   ))

g3 = go.FigureWidget(data=[],
                   layout=go.Layout(
                       title='Pressure',
                       xaxis_title='ecutwfc',
                       yaxis_title='relative diff(%)',
                   ))

In [49]:
def response(change):
    with g0.batch_update():
        g0.data = []
        g0.add_trace(plot_delta_hist(change['new']))
        
    with g1.batch_update():
        g1.data = []
        g1.add_traces(plot_convergence('convergence_cohesive_energy', change['new']))
        
    with g2.batch_update():
        g2.data = []
        g2.add_traces(plot_convergence('convergence_phonon_frequencies', change['new']))
        
    with g3.batch_update():
        g3.data = []
        g3.add_traces(plot_convergence('convergence_pressure', change['new']))

pcw.observe(response, names='output_pseudo_list')

In [51]:
widgets.VBox([g0, g1, g2, g3])

    'data': [],
    'layout': {'template': '...',
               'title': {'text…

Search and filter the results of verifications through Materials Cloud SSSP API.

User can also choose the protocol(critisize) to determine what kind of result they want to see, such as the precise one or the efficient one.

download the https://www.materialscloud.org/mcloud/api/v2/discover/sssp/elements/Si

The new API: https://app.swaggerhub.com/apis/unkcpz/mc-sssp/1.0.0